<h2>Face Recognition using Linear Discriminant Analysis</h2>


In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm_notebook as tqdm
import random

In [2]:
label = -1

train_images = []
train_labels = []

for i in tqdm(os.listdir('images/')):
    label+=1
    for nm in os.listdir('images/'+i):
        img = cv2.imread('images/'+i+'/'+nm, 0)
        img = np.ravel(img)
        train_images.append(img)
        train_labels.append(label)

c = list(zip(train_images, train_labels))
random.shuffle(c)
train_images, train_labels = zip(*c)

train_images = np.array(train_images)
train_labels = np.array(train_labels)
print(train_images.shape)
print(train_labels.shape)



(400, 14400)
(400,)


In [3]:

idx = int(len(train_images)*.7)
test_images = train_images[:][idx:]
test_labels = train_labels[idx:]

train_images = train_images[:][:idx]
train_labels = train_labels[:idx]

train_images = np.transpose(train_images)

print(train_images.shape)
print(test_labels.shape)

(14400, 280)
(120,)


In [4]:
train_mean = np.mean(train_images, axis=1)
train_mean = np.reshape(train_mean, (train_mean.shape[0], 1))
delta = train_images - train_mean
cov = np.cov(delta.T)
print(cov.shape)

(280, 280)


In [5]:
k = 70

eig_values, eig_vectors = np.linalg.eig(cov)
eig_values *= -1

idx = np.argpartition(eig_values, k)
ev = eig_vectors[idx[:k]]
print(ev.shape)

(70, 280)


In [6]:
eig_faces = np.dot(ev, delta.T)
print(eig_faces.shape)

sig = np.dot(eig_faces, delta)
print(sig.shape)

(70, 14400)
(70, 280)


<h3>Steps</h3>
<ol>
    <li> Dividing the data into class </li>
    <li> Calculation of mean </li>
</ol>


In [7]:
# Calculation of mean of each class and total mean

class_data = []
for i in range(0,40):
    tmp = []
    class_data.append(tmp)

for i in range(sig.shape[1]):
    label = train_labels[i]
    class_data[label].append(sig[:,i])

for i in range(0,40):
    class_data[i] = np.array(class_data[i])
class_data = np.array(class_data)


class_mean = []
for i in range(0,40):
    class_mean.append(np.mean(class_data[i], axis=0))

class_mean = np.array(class_mean)

total_mean = np.mean(sig, axis = 1)
total_mean = np.array(total_mean)
total_mean = np.reshape(total_mean, (total_mean.shape[0], 1))
    
    

<h3> Steps</h3>
<ol>
    <li> Calculate SW and SB matrix </li>
    <li> Criterion function </li>
</ol>

In [8]:
# Calculation of SW and SB

sw = np.zeros([70, 70])
for i in range(0,40):
    sw = sw + np.dot((class_data[i] - class_mean[i]).T, (class_data[i] - class_mean[i]))
print(f"Shape of SW : {sw.shape}")

sb = np.zeros([70, 70])
for i in range(0,40):
    sb = sb + np.dot((class_mean[i] - total_mean).T, (class_mean[i] - total_mean))
print(f"Shape of SB : {sb.shape}")

J = np.dot(np.linalg.inv(sw), sb)

Shape of SW : (70, 70)
Shape of SB : (70, 70)


<h3>Steps</h3>
<ol>
    <li> Eigen vector and Eigen values of the Criterion function </li>
    <li> Construct feature vectors </li>
   <li> Generate the fisher faces (FF) </li>
</ol>

In [9]:
# Best Principal Components

m = 40

eig_values, eig_vectors = np.linalg.eig(J)
print(f"Shape of eigen vectors : {eig_vectors.shape}")
eig_values *= -1

idx = np.argpartition(eig_values, m)
ev = eig_vectors[idx[:m]]
ev = ev.T
print(f"Shape of feature vectors: {ev.shape}")

Shape of eigen vectors : (70, 70)
Shape of feature vectors: (70, 40)


In [10]:
# Generating Fisher Faces {FF}

fisher_face = np.dot(ev.T, sig)
print(f"Shape of FF : {fisher_face.shape}")

Shape of FF : (40, 280)


In [11]:
# Testing
count = 0
for j in tqdm(range(test_labels.shape[0])):
    image = test_images[j]
    image = image - np.mean(train_images, axis=1)
    proj_test_face = np.dot(eig_faces, image)
    proj_fisher_face = np.dot(ev.T, proj_test_face)
    
    dist = []
    for i in range(fisher_face.shape[1]):
        dist.append(np.linalg.norm(proj_fisher_face - fisher_face[:,i]))
    
    index = np.where(dist == np.amin(dist))
    index = index[0]
    
    if train_labels[index] == test_labels[j]:
#         plt.subplot(1,2,1)
#         img = np.reshape(train_images[:,index], [120, 120])
#         plt.imshow(img, cmap='gray')
#         plt.subplot(1,2,2)
#         img = np.reshape(test_images[j], [120, 120])
#         plt.imshow(img, cmap='gray')
#         plt.show()
        count+=1

print(f"Accuracy : {count*100/ test_labels.shape[0]} at k = {70} and m = {40}")


Accuracy : 91.66666666666667 at k = 70 and m = 40
